# Webscrapping - Selenium 

La doc : https://selenium-python.readthedocs.io/navigating.html<br>
cheatsheet : http://allselenium.info/python-selenium-commands-cheat-sheet-frequently-used/<br>
moins complète mais plus propre : http://akul.me/blog/2016/selenium-cheatsheet/ <br>

## Installation 

Avec Anaconda : ouvrir anaconda prompt et executer : "conda install -c conda-forge selenium" 
Avec juste python, télécharger geckodrive, le mettre dans le path ou au même endroit que votre jupyter notebook
https://github.com/mozilla/geckodriver/releases

On commence par importer les modules dont on aura besoin. Le webdriver est l’objet qui gère le navigateur utilisé par Selenium. La classe “Keys” permet d’utiliser les touches du clavier. Et la fonction sleep pour mettre le scripte en “pause” pour les chargement de page

On utilise le driver Firefox mais il en existe d’autres celui de Chrome et PhantomJS par exemple.

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#driver = webdriver.Chrome()
browser = webdriver.Firefox()
browser.get('https://m.europages.fr/')

In [3]:
# On crée une instance du web-driver Firefox et on va sur la page de eBay.fr
#driver = webdriver.Chrome()
#browser = webdriver.Firefox(executable_path="C:\\Users\\pierremarie\\Desktop\\NOTEBOOK_data_ia\\geckodriver.exe")
#browser.get('https://m.europages.fr/')
#browser.maximize_window()

### Différentes manières de sélectionner un élement

In [4]:
input_searche_s = browser.find_element_by_class_name('querystring')
input_searche_s

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="cef4c927-995e-4439-b22a-31d3bc9931ee", element="e716e3aa-2274-4ad8-af20-8c0aea56e926")>

In [5]:
#browser.find_element_by_xpath('//*[@id="querystring"]').send_keys('expoys')

In [6]:
#browser.find_element_by_id('querystring').clear()

In [7]:
input_searche_s.send_keys('carbone')

### Cliquer sur un élement

In [8]:
buttun_s = browser.find_element_by_class_name('arrow-zoom')
buttun_s.click()

In [9]:
browser.back()

### Relancer une nouvelle recherche avec pour mots clés : "batiment".

### Récupérer le nom de la première entreprise.

### Cliquer sur le premier élément de la liste des résultats

### Récupérer le numéro de TVA de l'entreprise (Il sera nécessaire d'utiliser une expression régulière)

### Récupérer le pays d'exercice de l'entreprise

### Récupérer le descriptif de l'entreprise

### Récupérer le site de l'entreprise

### Récupérer l'adressse de l'entreprise

In [11]:
from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

import re

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select

# Multi
from multiprocessing import Pool, cpu_count

# Time
import datetime
import time

# Tools
from itertools import repeat
import unittest
from unittest.mock import patch

In [12]:
# -------------------------------
# ---------- FONCTIONS ----------
# -------------------------------

def turn_page ():
    try:
        next_page_button = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/form/div[2]/div[2]/div[2]/div/ul/li[7]/a')
        next_page_button.click()
    except(NoSuchElementException):
        print ("Il n'y a plus de pages à tourner")
        
# dans la page de n resultats, recuperer les liens de chaque résultat -> liste de liens
def get_result_urls(sleep_time, resultats_par_page):
    url_list = []
    counter = 0
    for index in range(3,resultats_par_page+2):
        try:
            time.sleep(sleep_time)
            url_list.append(browser.find_element_by_xpath(f'/html/body/div[2]/div/div[1]/form/div[2]/div[{index}]/article/div/div[1]/div/a').get_attribute('href'))
            time.sleep(sleep_time)
            counter += 1
        except:
            print("Il n'y a plus d'url à récupérér!!!! Itérations effectuées : ", counter)
            break
    return url_list

# ouvrir url dans nouvelle fenêtre, executer get_attributes, remplir liste, fermer onglet -> liste complétée
def open_url_get_attributtes_go_back(sleep_time,liste_a_completer):
    counter = 0
    # for url in url_list[0:20]:
    for url in url_list:
        try:
            main_window = browser.current_window_handle
            # Open a new tab in blank
            browser.execute_script("window.open(''),'_blannk'")
            # Switch to the new window
            browser.switch_to.window(browser.window_handles[1])

            # Change the url in the .get**
            browser.get(url)
            # Completer la liste
            attributs_voiture = get_attributes(beke_occasion_xpath)
            liste_a_completer.append(attributs_voiture)
            # Temps d'attente
            time.sleep(sleep_time)
            # Close Current Tab
            browser.close()
            # Focus to the main window
            browser.switch_to.window(main_window)
            counter += 1
        
        except:
            print("Il n'y a plus d'url dans la liste. J'ai fini ! Itérations : ", counter)
            break
        
    return liste_a_completer

def get_attributes(dictionnaire):
    liste = []
    for item in dictionnaire.items() : 
                try:
                     liste.append(browser.find_element_by_xpath(item[1]).text)

                except (NoSuchElementException) : 
                    liste.append('nan')
    return liste

# VOITURES D'OCCASION SI OUVERT DANS NOUVEL L'ONGLET 
beke_occasion_xpath = {
    "nom":'/html/body/div[1]/div[1]/div/div/div[2]/h3',    
    "sous_nom":'/html/body/div[1]/div[1]/div/div/div[2]/span',
    "ref":"/html/body/div[1]/div[1]/div/div/div[3]/div[1]/div[2]/div",
    "marque":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[1]/a',
    "modele":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[2]/a',
    "categorie":'/html/body/div[1]/div[1]/div/div/div[1]/ul/li[3]/a',
    "co2":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]/div/div[2]',    
    "nb_km":"//td[preceding::th[text()[contains(.,'Kilométrage')]]]",    
    "carburant":'/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div/div[2]/div[1]/div[2]/span',    
    "boite_vitesses":"//span[preceding::label[text()[contains(.,'Boite')]]]",    
    "couleur":"//td[preceding::th[text()[contains(.,'Couleur')]]]",
    "nb_portes":"//td[preceding::th[text()[contains(.,'Nombre de portes')]]]",
    "nb_places":"//td[preceding::th[text()[contains(.,'Nombre de places')]]]",
    "mise_circulation":"//td[preceding::th[text()[contains(.,'circulation')]]]",    
    "conso_mixte":"//td[preceding::th[text()[contains(.,'Conso. mixte')]]]",    
    "p_fiscale":"//td[preceding::th[text()[contains(.,'Puissance fiscale')]]]",    
    "p_din":"//td[preceding::th[text()[contains(.,'Puissance DIN')]]]",    
    "critair":"//th[text()[contains(.,'Certificat')]]",    
    "prix":'/html/body/div[1]/div[1]/div/aside/div/div[1]/div[1]/div[1]/div/span',     
}

# -------------------------------
# ---------- SCRAPPING ----------
# -------------------------------

# HEADLESS
#options = webdriver.ChromeOptions()
#options.add_argument('headless')
#browser = webdriver.Chrome(options=options)
browser = webdriver.Firefox()
browser.get('http://www.bekeautocenter.fr/catalogue/?search[vnvo]=1&search[marques][0]=Citro%C3%ABn&search[marques][1]=Peugeot&search[marques][2]=Renault&search[minPrice]=0&search[maxPrice]=40000&search[order]=marque|asc&search[limit]=100&search[_token]=S90FBTV8BwR2bWGAgT4fdtttoPj4ZPTKrUYdM27skPg&_=1590561915186&page=1')


# browser = webdriver.Chrome()
# browser.maximize_window()
# browser.get('http://www.bekeautocenter.fr/catalogue/?search[vnvo]=1&search[marques][0]=Citro%C3%ABn&search[marques][1]=Peugeot&search[marques][2]=Renault&search[minPrice]=0&search[maxPrice]=40000&search[order]=marque|asc&search[limit]=100&search[_token]=S90FBTV8BwR2bWGAgT4fdtttoPj4ZPTKrUYdM27skPg&_=1590561915186&page=1')

# Nombre total de résultats au format : '816 RÉFÉRENCES' (type = str)
beke_total_results = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/form/div[1]/div[2]/h3').text
print("Nombre total de résultats :", beke_total_results)

# Dernière page disponible au format : 9 (type = int)
beke_last_page = int(re.findall("(\d+) ",beke_total_results)[0]) // 100 + 1
print("Dernière page disponible : ", beke_last_page)

liste_a_completer = []

for page in range(1,beke_last_page+1):
    
    # Liste de liens de la page en cours :
    url_list = get_result_urls(0.5, 100)

    # Va dans chaque lien et recupere les infos de chaque voiture
    liste_a_completer = open_url_get_attributtes_go_back(10,liste_a_completer)

    # On tourne la page :
    turn_page() 
    
    print("Echantillons recupérés : ",len(liste_a_completer))

Nombre total de résultats : 434 RÉFÉRENCES
Dernière page disponible :  5
Echantillons recupérés :  99
Echantillons recupérés :  198
Echantillons recupérés :  297
Echantillons recupérés :  396
Il n'y a plus d'url à récupérér!!!! Itérations effectuées :  39
Il n'y a plus de pages à tourner
Echantillons recupérés :  435


In [13]:
# Convertir liste de listes en DataFrame
column_names = [item[0] for item in beke_occasion_xpath.items()]

df = pd.DataFrame.from_records(liste_a_completer,columns=column_names)
df_copy = df
df.shape

(435, 19)

In [14]:
df.to_csv("beke.csv", index=False)

In [15]:
def preprocess_beke_dataframe (df):
    # Enlever les espaces entre chiffres
    df['prix'] = df['prix'].str.replace(' ', '', regex=True)
    df['p_din'] = df['p_din'].str.replace(' ', '', regex=True)
    df['ref'] = df['ref'].str.replace(' - ', '', regex=True)

    # Remplacer nan par '0' pour pouvoir convertir en nombre plus tard
    df['conso_mixte'] = df['conso_mixte'].str.replace('nan', '0', regex=True)

    # Extraire par regex puis changer le type
    for col in ['co2', 'nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']:
        df[col] = df[col].str.extract(r'(\d+)').astype(int)

    # Extraire par regex puis changer le type (colonnes difficiles)
    df['conso_mixte'] = df['conso_mixte'].str.extract('(\d+\.?)L').astype(float)
    df['ref'] = df['ref'].str.extract('(\d+)').astype(float)
    return df